In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from citipy import citipy
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'citipy'

In [ ]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lats_lngs

In [ ]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [ ]:
# Create a list for holding the cities.
cities = []

# Identify nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
# Import the requests library.
import requests

# Import the datetime module from the datetime library.
from datetime import datetime

# Import the API key.
from config import weather_api_key

In [ ]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

### Retrieve the following information from the API call.
    - The latitude and longitude
    - The Max temperature
    - The % humidity
    - The % cloudiness
    - The Wind speed
    - The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 
    - If it is raining, the amount of rain in inches for the last hour (1 hr). If not raining = "0" inches.
    - If it is snowing, the amount of snow in inches for the last hour (1 hr). If not snowing = "0" inches
----
Hint: Use try / except blocks to get the amount of rain or snow. 

In [ ]:
# List of city data
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1

    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"] 
        try:
            city_rain_inches = city_weather["rain"]["1h"]
        except KeyError:
            city_rain_inches = 0
        try:
            city_snow_inches = city_weather["snow"]["1h"]
        except KeyError:
            city_snow_inches = 0        
            
            # Append the City information into city_data list
        city_data.append({"City": city.title(), 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Rain (inches)": city_rain_inches,
                          "Snow (inches)": city_snow_inches,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})
    # If an error is experienced, skip the city

    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
len(city_data)

In [ ]:
# Convert array of dictionaries to a pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [ ]:
city_data_df.head(10)

In [ ]:
city_data_df.columns

In [ ]:
# Reorder the columns in the order you want them to appear.
new_column_order = ["City", "Country", "Lat",
                    "Lng", "Max Temp", "Humidity", 
                    "Cloudiness", "Wind Speed", 
                    "Current Description", "Rain (inches)","Snow (inches)"] 

# Assign a new or the same DataFrame the new column order.
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

In [ ]:
city_data_df.dtypes

In [ ]:
raining_cities = city_data_df.loc[city_data_df['Rain (inches)']>0]
raining_cities

In [ ]:
snowing_cities = city_data_df.loc[city_data_df['Snow (inches)']>0]
snowing_cities

In [ ]:
# Create the output File (CSV)
output_data_file = "data/WeatherPy_Database.csv"
# Export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")